Wichtige Infos: Bitte den nachstehenden Codeblock ausführen, um die einzelnen User-Storys bzw. codes, ausführen zu können. 

Änderungen wie das Erstellen einer Buchung, das Generieren einer Rechnung oder das Stornieren einer Buchung wirken sich direkt auf die Datenbank aus.
Um dies zu überprüfen, kann die Datenbank nach der Ausführung des entsprechenden Codes heruntergeladen und in einem externen Tool wie SQLonline oder DB Browser for SQLite geöffnet werden.
Aufgrund des zeitlichen Rahmens haben wir bewusst darauf verzichtet, zusätzliche Kontroll-SQL-Abfragen in das Notebook zu integrieren.

In [1]:
import shutil
from pathlib import Path
import os

from datetime import date, datetime

original_db = Path("./database/hotel_reservation_sample.db")
working_db = Path("./database/working.db")

shutil.copyfile(str(original_db), str(working_db))

os.environ['DB_FILE'] = str(working_db)

from businessLogic.addressManager import AddressManager
from businessLogic.bookingManager import BookingManager
from businessLogic.facilityManager import FacilityManager
from businessLogic.hotelManager import HotelManager
from businessLogic.invoiceManager import InvoiceManager
from businessLogic.roomManager import RoomManager
from businessLogic.roomTypeManager import RoomTypeManager
from businessLogic.guestManager import GuestManager
from businessLogic.userManager import UserManager


address_manager = AddressManager()
booking_manager = BookingManager()
facility_manager = FacilityManager()
hotel_manager = HotelManager()
invoice_manager = InvoiceManager()
room_manager = RoomManager()
room_type_manager = RoomTypeManager()
guest_manager = GuestManager()
user_manager = UserManager()


User Story 1.1: "Ich möchte alle Hotels in einer Stadt durchsuchen, damit ich das Hotel nach meinem bevorzugten Standort (Stadt) auswählen kann."

Beschreibung:
Der Benutzer gibt eine Stadt ein, in der er ein Hotel suchen möchte. Der Input wird an den hotel_manager übergeben, welcher über hotelAccess auf die Datenbank zugreift und alle Hotels in der gewünschten Stadt zurückgibt.
Die Namen der gefundenen Hotels werden anschliessend ausgegeben.

Überlegungen:
In der DB gibt es keine direkte Information zur Stadt, in welcher sich das Hotel befindet, also müssen wir über einen JOIN auf die Tabelle Address zugreifen, damit wir die Stadt abfragen können.

In [ ]:
city = input("Wo möchten Sie nach Hotels suchen?")

hotels = hotel_manager.search_hotels_by_city(city,)
for hotel in hotels:
    print(f'Hotels in {city}:')
    print(hotel.name)

User Story 1.2: Ich möchte alle Hotels in einer Stadt nach der Anzahl der Sterne (z.B. mindestens 4 Sterne) durchsuchen.

Beschreibung: Der Benutzer gibt eine Stadt und die Anzahl Sterne, die das Hotel mindestens haben soll, ein. Die Eingaben werden an den hotel_manager übergeben, welcher über die Methode search_hotels_by_city_and_stars auf die DB zugreift und alle Hotels mit den passenden Kriterien zurückgibt. 

Überlegungen: Da wir nach einer Mindestanzahl suchen, ist es wichtig, dass wir >= (grösser gleich) verwenden, da ansonsten falsche Ergebnisse angezeigt werden. Zusätzlich legen wir fest, dass der Input eine Zahl sein muss.

In [ ]:
city = input("Wo möchten Sie nach Hotels suchen?")
stars = int(input("Wieviele Sterne soll das Hotel haben?"))

hotels = hotel_manager.search_hotels_by_city_and_stars(city, stars)
for hotel in hotels:
    print(f'Hotels in {city} mit mind. {stars} Sternen:')
    print(hotel.name)

User-Story 1.3: Ich möchte alle Hotels in einer Stadt durchsuchen, die Zimmer haben, die meiner Gästezahl entsprechen.

Beschreibung: Der Benutzer gibt eine Stadt und die gewünschte Anzahl an Gäste, für welche er ein Zimmer benötigt, ein. Die Eingaben werden an den hotel_manager übergeben, welcher über die Methode search_hotels_by_city_and_max_guests auf die DB zugreift und alle Hotels mit   den passenden Kriterien zurückgibt. 

Überlegungen: Hier können wir auch nicht direkt über die Tabelle Hotel auf die Information bezüglich der maximalen Gästeanzahl pro Zimmer zugreifen, dass heisst: Wir müssen wieder über die Tabellen Room und Room_Type (type_id) mittels JOIN auf die Informationen zugreifen. Auch hier verwenden wir wieder >= (grösser gleich) und eine Zahl als Input.

In [ ]:
city = input("Wo möchten Sie nach Hotels suchen?")
max_guests = int(input("Für vieviele Gäste benötigen Sie ein Zimmer?"))

hotels = hotel_manager.search_hotels_by_city_and_max_guests(city, max_guests)
for hotel in hotels:
    print(f'Hotels in {city} mit Zimmern für {max_guests} Personen:')
    print(hotel.name)

User-Story 1.4: Ich möchte alle Hotels in einer Stadt durchsuchen, die während meines Aufenthaltes ("von" (check_in_date) und "bis" (check_out_date)) Zimmer zur Verfügung haben, damit ich nur relevante Ergebnisse sehe.

Beschreibung: Der Benutzer gibt eine Stadt sowie ein Check-in- und ein Check-out-Datum ein. Zuerst werden über den hotel_manager alle Hotels in der gewünschten Stadt geladen. Anschliessend wird über den room_manager mit der Methode find_available_rooms_for_guest nach Zimmern gesucht, die im angegebenen Zeitraum nicht belegt sind. Aus den verfügbaren Zimmern lassen sich dann die dazugehörigen Hotels ableiten, welche für den Zeitraum Zimmer frei haben. Am Schluss wird dem Benutzer die Liste dieser Hotels angezeigt.

Überlegungen: Da keine direkte Information zur Verfügbarkeit in der Room Tabelle vorhanden ist, muss geprüft werden, ob es keine bestehende Buchung im gewünschten Zeitraum gibt.
Dazu verwendet die SQL-Abfrage im roomAccess.find_available_rooms eine Negativbedingung (NOT IN), die alle Zimmer ausschliesst, welche zu dem Zeitraum bereits gebucht (und nicht storniert) sind.
Die Bedingung (check_in < ? AND check_out > ?) sorgt dabei dafür, dass alle überlappenden Buchungen ausgeschlossen werden.


In [10]:
city = input("Wo möchten Sie nach Hotels suchen?")
check_in_date = input("Check-in Datum (YYYY-MM-DD): ")
check_out_date = input("Check-out Datum (YYYY-MM-DD): ")

hotels = hotel_manager.search_hotels_by_city(city)
rooms_data = room_manager.find_available_rooms_for_guest(check_in_date, check_out_date)

for hotel in hotels:
    print(f'Hotels in {city} verfügbar vom {check_in_date} bis {check_out_date}:')
    print(hotel.name)

Hotels in Zürich verfügbar vom 2025-11-11 bis 20225-12-12:
Hotel Baur au Lac


User-Story 1.5: Ich möchte Wünsche kombinieren können, z.B. die verfügbaren Zimmer zusammen mit meiner Gästezahl und der mindest Anzahl Sterne.

Beschreibung: Der Benutzer kann mittels Inputs zu Stadt, Sterne, Gäste usw. nach Hotels suchen, welche seinen Kriterien entsprechen. Hier werden die Methoden und Überlegungen der vorherigen User-Stories angewendet.

In [ ]:
city = input("Wo möchten Sie nach Hotels suchen?")
stars = int(input("Wieviele Sterne soll das Hotel haben?"))
max_guests = int(input("Für vieviele Gäste benötigen Sie ein Zimmer?"))
check_in_date = input("Check-in Datum (YYYY-MM-DD): ")
check_out_date = input("Check-out Datum (YYYY-MM-DD): ")

hotels = hotel_manager.search_hotels_by_city_and_stars(city, stars)
hotels = hotel_manager.search_hotels_by_city_and_max_guests(city, max_guests)
rooms_data = room_manager.find_available_rooms_for_guest(check_in_date, check_out_date)

for hotel in hotels: 
    print(f'Hotels in {city} mit {stars} Sternen und Kapazität für {max_guests} Gäste verfügbar vom {check_in_date} bis {check_out_date}:')
    print(hotel.name)

User-Story 1.6: Ich möchte die folgenden Informationen pro Hotel sehen: Name, Adresse, Anzahl der Sterne.

Beschreibung: Dem Benutzer werden zuerst alle verfügbaren Hotels aus der Datenbank angezeigt, nummeriert in einer Liste. Anschliessend kann der Benutzer ein Hotel auswählen. Nach der Auswahl zeigt das System die vollständigen Informationen zum gewählten Hotel an: Hotelname, Hotel Adresse usw.

Überlegungen: Die Hotels werden in einer Liste mit enumerate ausgegeben, damit sie nummeriert sind und der Benutzer sie einfach anhand eines Indexes auswählen kann. Da Listen in Python bei 0 beginnen, der Benutzer aber eine Zahl ab 1 eingibt, wird selected_index minus 1 gerechnet, um den korrekten Listenindex zu treffen. Vor dem Zugriff auf das gewählte Hotel wird geprüft, ob der eingegebene Index innerhalb der gültigen Liste liegt, so werden ungültige Eingaben abgefangen und eine entsprechende Fehlermeldung ausgegeben.


In [ ]:
hotels = hotel_manager.search_all_hotels()

print("Verfügbare Hotels:\n")
for index, hotel in enumerate(hotels):
    print(f"{index + 1}. {hotel.name}")

selected_index = int(input("Bitte ein Hotel auswählen: ")) - 1

if 0 <= selected_index < len(hotels):
    selected_hotel = hotels[selected_index]
    print(f"\nHotel Information für: {selected_hotel.name}")
    print(f"Adresse: {selected_hotel.address.street}, {selected_hotel.address.zip_code} {selected_hotel.address.city}")
    print(f"Sterne: {selected_hotel.stars}")
else:
    print("Ungültige Eingabe.")


User-Story 2.1: Ich möchte die folgenden Informationen pro Zimmer sehen: Zimmertyp, max. Anzahl der Gäste, Beschreibung, Ausstattung, Preis pro Nacht und Gesamtpreis.


In [ ]:
room_id = int(input("Bitte Zimmer-ID eingeben: "))
check_in_str = input("Check-in Datum (YYYY-MM-DD): ")
check_out_str = input("Check-out Datum (YYYY-MM-DD): ")

details = room_manager.get_room_details_with_total_price(room_id, check_in_str, check_out_str)
if details:
    print(f"Zimmer {details['room_number']} (ID: {details['room_id']})")
    print(f"Zimmertyp: {details['room_type']['description']} (max. Gäste: {details['room_type']['max_guests']})")
    print(f"Ausstattung: {', '.join(details['facilities']) if details['facilities'] else 'Keine'}")
    print(f"Preis pro Nacht (Standard): {details['price_per_night']} CHF")
    print(f"Gesamtpreis für Aufenthalt (dynamisch): {details['total_price']} CHF")
else:
    print("Zimmer nicht gefunden.")

User-Story 2.2: Ich möchte nur die verfügbaren Zimmer sehen, sofern ich meinen Aufenthalt (von – bis) spezifiziert habe.

In [17]:
check_in_str = input("Check_in Datum(YYYY-MM-DD): ")
check_out_str = input("Check-out (YYYY-MM-DD): ")

try:
    check_in = datetime.strptime(check_in_str, "%Y-%m-%d").date()
    check_out = datetime.strptime(check_out_str, "%Y-%m-%d").date()
except ValueError:
    print("Falsches Datum Format. Bitte verwenden Sie YYYY-MM-DD.")
    check_in = None
    check_out = None

if check_in and check_out:
    available_rooms = room_manager.find_available_rooms_for_guest(
        check_in_date=check_in,
        check_out_date=check_out
    )

    if available_rooms:
        print(f"Verfügbare Zimmer vom {check_in} bis {check_out}:")
        for room in available_rooms:
            print(
                f"Hotel: {room.hotel.name}, "
                f"Stadt: {city}, "
                f"Room ID: {room.room_id}, "
                f"Room Number: {room.room_number}"
            )
    else:
        print(f"Keine verfügbaren Zimmer zwischen dem {check_in} und {check_out} gefunden.")

Verfügbare Zimmer vom 2025-11-11 bis 2025-12-12:
Hotel: 5, Stadt: Zürich, Room ID: 1, Room Number: 101
Hotel: 5, Stadt: Zürich, Room ID: 2, Room Number: 102
Hotel: 5, Stadt: Zürich, Room ID: 3, Room Number: 201
Hotel: 5, Stadt: Zürich, Room ID: 4, Room Number: 301
Hotel: 5, Stadt: Zürich, Room ID: 5, Room Number: 401


User-Story 3.1: Ich möchte neue Hotels zum System hinzufügen

In [ ]:
username = input("Enter your username: ") # The username is "Admin"
password = input("Enter password: ") # The password is "FHNW"

if user_manager.is_admin(username, password):
    street = input("Strasse und Hausnummer: ")
    city = input("Stadt: ")
    zip_code = input("PLZ: ")

    address = address_manager.add_address(street, city, zip_code)
    if address:
        print(f"Adresse erfolgreich angelegt: {street}, {zip_code} {city}")
        address_id = address[0]
    else:
        print("Fehler beim Anlegen der Adresse.")
        address_id = int(input("Bitte Address-ID manuell eingeben: "))

    hotel_name = input("Hotelname: ")
    stars = int(input("Sterne (1-5): "))

    hotel = hotel_manager.add_hotel(hotel_name, stars, address_id)
    if hotel:
        print(f"Hotel '{hotel_name}' erfolgreich angelegt!")
        hotel_id = hotel["hotel_id"]
    else:
        print("Fehler beim Anlegen des Hotels.")
        hotel_id = int(input("Bitte Hotel-ID manuell eingeben: "))
        
    room_number = input("Zimmernummer: ")
    room_type_id = int(input("Room-Type-ID: "))
    price_per_night = float(input("Preis pro Nacht: "))

    room = room_manager.add_room(room_number, hotel_id, room_type_id, price_per_night)
    if room:
        print(f"Zimmer {room_number} erfolgreich angelegt!")
    else:
        print("Fehler beim Anlegen des Zimmers.")

else:
    print("Sie haben keine Zugriffsrechte.")

User-Story 3.2: Ich möchte Hotels aus dem System entfernen

In [ ]:
username = input("Enter your username: ") # The username is "Admin"
password = input("Enter password: ") # The password is "FHNW"

if user_manager.is_admin(username, password):
    # The first inputs need to be strings, because they are optional and can be left empty. If the are an integer they will make an error if left empty.
    # Therefore we convert them to integers at a later point. 
    address_id_str = input("Insert the address_id from the address you want to delete (leave empty if not): ") # address_ id as a string
    room_id_str = input("Insert the room_id from the Room you want to delete (leave empty if not): ") # room_id as a string
    hotel_id_str = input("Insert the hotel_id from the Hotel you want to delete (leave empty if not): ") # hotel_id as a string

    address_id = int(address_id_str) if address_id_str else None # converting string to an integer
    room_id = int(room_id_str) if room_id_str else None # converting string to an integer
    hotel_id = int(hotel_id_str) if hotel_id_str else None # converting string to an integer

    if address_id:
        result = address_manager.delete_address(address_id)
        if result:
            print(f"Adress with address_id {address_id} deletded!")
        else:
            print(f"Adress with address_id {address_id} couldn't be deleted.")

    if room_id:
        result = room_manager.delete_room(room_id)
        if result: 
            print(f"Zimmer with room_id {room_id} deleted!")
        else:
            print(f"Zimmer with room_id {room_id} couldn't be deleted.")

    if hotel_id:
        result = hotel_manager.delete_hotel(hotel_id)
        if result:
            print(f"Hotel with hotel_id {hotel_id} deleted!")
        else:
            print(f"Hotel with hotel_id {hotel_id} couldn't be deleted.")

    if not (address_id or room_id or hotel_id):
        print("No ID input. Nothing deleted.")

else:
    print("Sie haben keine Zugriffsrechte.")

User-Story 3.3: Ich möchte die Informationen bestimmter Hotels aktualisieren, z.B. den Namen, die Sterne usw. 

In [ ]:
username = input("Enter your username: ") # The username is "Admin"
password = input("Enter password: ") # The password is "FHNW"

if user_manager.is_admin(username, password):
    # Here it is possible to choose which thing/entity we want to change.
    print("What do you want to change?")
    print("1: Hotel")
    print("2: Adress")
    print("3: Room")
    choose = input("Choose between 1 and 3: ")

    if choose == "1":
        hotel_id_str = input("hotel_id: ") # hotel_id as a string, because otherwise it would make an error if left empty (this is the same for the next inputs)
        if not hotel_id_str:
            print("No hotel_id inserted, actualization ended.")
        else:
            hotel_id = int(hotel_id_str)
            name = input("if you want to change the name insert new name otherwise leave empty: ")
            stars_str = input("new number of stars (1-5), otherwise leave empty: ")
            address_id_str = input("new address_id, otherwise leave empty: ")
            stars = int(stars_str) if stars_str else None
            address_id = int(address_id_str) if address_id_str else None

            result = hotel_manager.update_hotel(
                hotel_id,
                name=name if name else None,
                stars=stars,
                address_id=address_id
            )
            if result:
                print("Hotel changed succesfully!")
            else:
                print("Hotel changes couldn't be made.")

    elif choose == "2":
        address_id_str = input("address_id: ")
        if not address_id_str:
            print("no address_id inserted, actualization ended.")
        else:
            address_id = int(address_id_str)
            street = input("new street name and number, otherwise leave empty: ")
            city = input("new city name, otherwise leave empty: ")
            zip_code_str = input("new zip code, otherwise leave empty: ")
            zip_code = int(zip_code_str) if zip_code_str else None

            result = address_manager.update_address(
                address_id,
                street=street if street else None,
                city=city if city else None,
                zip_code=zip_code
            )
            if result:
                print("Address changed succesfully!")
            else:
                print("Address changes couldn't be made.")

    elif choose == "3":
        room_id_str = input("room_id: ")
        if not room_id_str:
            print("no room_id inserted, actualization ended.")
        else:
            room_id = int(room_id_str)
            room_number = input("new room number, otherwise leave empty: ")
            hotel_id_str = input("new hotel_id, otherwise leave empty: ")
            room_type_id_str = input("new room_type_id, otherwise leave empty: ")
            price_per_night_str = input("new price per night, otherwise leave empty: ")

            hotel_id = int(hotel_id_str) if hotel_id_str else None
            room_type_id = int(room_type_id_str) if room_type_id_str else None
            price_per_night = float(price_per_night_str) if price_per_night_str else None

            result = room_manager.update_room(
                room_id,
                room_number if room_number else None,
                hotel_id,
                room_type_id,
                price_per_night
            )
            if result:
                print("Room changed succesfully!")
            else:
                print("Room changes couldn't be made.")
    else:
        print("invalid input.")

else:
    print("Sie haben keine Zugriffsrechte.")

User-Story 4: Als Gast möchte ich ein Zimmer in einem bestimmten Hotel buchen, um meinen Urlaub zu planen.

Beschreibung: Der Benutzer wählt mittels der Funktion choose_option das gewünschte Hotel, Zimmer und den Gast (sich selbst) aus der entsprechenden Liste aus. Hierfür werden insgesamt 3 verschiedene managers verwendet: hotel_manager, room_manager und guest_manager.
Am Schluss wird eine entsprechende Buchung im ausgewählten Hotel erstellt und zu

Überlegungen: Diese User Story stellt sich als komplexer heraus als zuerst gedacht, da wir mehrere manager bzw. Methoden verwenden müssen, die am Schluss zum gewünschten Ergebnis führen. Beim User Input wurde darauf geachtet, dass es so benutzerfreundlich wie möglich ist, daher auch index + 1 bzw. choice -1. Zusätzlich wurde darauf geachtet, dass bei Fehleingaben oder nicht verfügbaren Zimmern für den gewünschten Zeitraum, eine entsprechende Meldung ausgegeben wird. Für diese und andere User-Stories wurde zusätlich der guest_access und guestManager erstellt, damit sich der Benutzer selbst oder jemand anderes auswählen kann. Nach dem Buchen kommt eine Meldung, welche dem Benutzer die Buchung bestätigt und die dazugehörige Buchungs-ID mitteilt, um diese auch für spätere Zwecke (Buchung stornieren etc.) benutzen zu können.

In [ ]:
def choose_option(options, label_fn, prompt_text):
    while True:
        for i, item in enumerate(options):
            print(f"{i+1}. {label_fn(item)}")
        try:
            choice = int(input(f"{prompt_text} (Zahl eingeben): "))
            if 1 <= choice <= len(options):
                return options[choice - 1]
            else:
                print("Ungültige Zahl. Bitte versuch es erneut.")
        except ValueError:
            print("Ungültige Eingabe. Bitte eine Zahl eingeben.")

# Hotelwahl
print("Wähle ein Hotel:")
hotel = choose_option(hotel_manager.search_all_hotels(), lambda h: h.name, "Wähle ein Hotel")

# Check-in und Check-out
check_in = datetime.strptime(input("Check-in (YYYY-MM-DD): "), "%Y-%m-%d").date()
check_out = datetime.strptime(input("Check-out (YYYY-MM-DD): "), "%Y-%m-%d").date()

# Verfügbare Zimmer suchen
rooms = room_manager.find_available_rooms_for_guest(check_in, check_out, city=hotel.address.city)
if not rooms:
    print("Keine verfügbaren Zimmer im gewünschten Zeitraum.")
    exit()

# Zimmerauswahl
print("Wähle ein verfügbares Zimmer:")
room = choose_option(rooms, lambda r: f"Zimmer {r.room_number} (ID: {r.room_id})", "Wähle ein Zimmer")
details = room_manager.get_room_details_with_total_price(room.room_id, str(check_in), str(check_out))
amount = (check_out - check_in).days * details["price_per_night"]

# Gastauswahl
print("Wähle einen Gast:")
guest = choose_option(guest_manager.search_all_guests(), lambda g: f"{g.first_name} {g.last_name}", "Wähle einen Gast")

# Buchung erstellen
booking_id = booking_manager.create_booking(guest.guest_id, room.room_id, str(check_in), str(check_out), amount)

print(f"Buchung erfolgreich! Buchungs-ID: {booking_id}")

User-Story 5: Als Gast möchte ich nach meinem Aufenthalt eine Rechnung erhalten, damit ich einen Zahlungsnachweis habe.

Beschreibung:
Der Benutzer wählt mithilfe der Funktion choose_option zunächst einen Gast aus, für den eine Rechnung erstellt werden soll. Anschliessend werden alle zugehörigen Buchungen des Gasts mittels booking_manager.search_booking_by_guest geladen. Falls keine Buchungen vorhanden sind, wird dies dem Benutzer mitgeteilt. Ist mindestens eine Buchung vorhanden, kann er eine davon auswählen. Danach wird mit invoice_manager.get_invoice eine Rechnung für die gewählte Buchung erstellt.

Überlegungen:
Auch hier wurde die Funktion choose_option verwendet, um eine benutzerfreundliche und fehlerresistente Auswahl zu ermöglichen. Es wird sichergestellt, dass nur Buchungen angezeigt werden, die dem ausgewählten Gast tatsächlich zugeordnet sind. Zudem wird der aktuelle Stornostatus jeder Buchung direkt mit angezeigt, um Missverständnisse zu vermeiden. Eine zusätzliche Sicherheitsabfrage (j/n) verhindert versehentliche Stornierungen. Die finale Rückmeldung enthält eine klare Bestätigung oder Fehlermeldung. Technisch wurde dabei berücksichtigt, dass das Stornieren über ein einfaches Update der Spalte is_cancelled in der Datenbank erfolgt.

In [ ]:
def choose_option(options, label_fn, prompt_text):
    while True:
        for i, item in enumerate(options):
            print(f"{i+1}. {label_fn(item)}")
        try:
            choice = int(input(f"{prompt_text} (Zahl eingeben): "))
            if 1 <= choice <= len(options):
                return options[choice - 1]
            else:
                print("Ungültige Zahl. Bitte versuch es erneut.")
        except ValueError:
            print("Ungültige Eingabe. Bitte eine Zahl eingeben.")

print("Wähle einen Gast, für den du eine Rechnung erstellen willst:")
guest = choose_option(guest_manager.search_all_guests(), lambda g: f"{g.first_name} {g.last_name}", "Wähle einen Gast aus")

bookings = booking_manager.search_booking_by_guest(guest.guest_id)

if not bookings:
    print("Dieser Gast hat keine Buchungen.")
else:
    print("Wähle eine Buchung aus:")
    booking = choose_option(bookings, lambda b: f"Buchung {b.booking_id} vom {b.check_in_date} bis {b.check_out_date} - CHF {b.total_amount:.2f}", "Wähle eine Buchung aus")

    invoice_manager.get_invoice(booking.booking_id, booking.total_amount)
    print(f"Rechnung erfolgreich erstellt für Buchung {booking.booking_id} über CHF {booking.total_amount:.2f}.")

User-Story 6: Als Gast möchte ich meine Buchung stornieren, damit ich nicht belastet werde, wenn ich das Zimmer nicht mehr benötige.

Beschreibung: Zuerst wird mithilfe der Funktion choose_option ein Gast aus der Gästeliste ausgewählt. Danach werden alle zugehörigen Buchungen des Gastes mit booking_manager.search_booking_by_guest geladen. Falls keine Buchungen vorhanden sind, wird dies entsprechend ausgegeben. Gibt es Buchungen, kann der Benutzer eine davon auswählen, wobei auch angezeigt wird, ob die Buchung bereits storniert wurde. Nach einer zusätzlichen Sicherheitsabfrage (j/n) wird die Buchung über booking_manager.update_booking storniert, indem der Wert is_cancelled auf True gesetzt wird. Anschliessend wird eine Bestätigung oder eine Fehlermeldung ausgegeben.

Überlegungen: Auch hier wurde wieder die Funktion choose_option verwendet, um eine benutzerfreundliche und fehlerresistente Auswahl zu ermöglichen. Es wird sichergestellt, dass nur Buchungen angezeigt werden, die dem ausgewählten Gast tatsächlich zugeordnet sind. Zudem wird der aktuelle Stornostatus jeder Buchung direkt mit angezeigt, um Missverständnisse zu vermeiden. Eine zusätzliche Sicherheitsabfrage (j/n) verhindert versehentliche Stornierungen. Die finale Rückmeldung enthält eine klare Bestätigung oder Fehlermeldung. Technisch wurde dabei berücksichtigt, dass das Stornieren über ein einfaches Update der Spalte is_cancelled in der Datenbank erfolgt. 

In [ ]:
def choose_option(options, label_fn, prompt_text):
    while True:
        for i, item in enumerate(options):
            print(f"{i+1}. {label_fn(item)}")
        try:
            choice = int(input(f"{prompt_text} (Zahl eingeben): "))
            if 1 <= choice <= len(options):
                return options[choice - 1]
            else:
                print("Ungültige Zahl. Bitte versuch es erneut.")
        except ValueError:
            print("Ungültige Eingabe. Bitte eine Zahl eingeben.")

# 1. Alle Gäste anzeigen und Gast auswählen
guests = guest_manager.search_all_guests()
print("Wähle einen Gast:")
selected_guest = choose_option(guests, lambda g: f"{g.first_name} {g.last_name}", "Wähle einen Gast aus")

# 2. Alle Buchungen dieses Gastes anzeigen
bookings = booking_manager.search_booking_by_guest(selected_guest.guest_id)
if not bookings:
    print("Dieser Gast hat keine Buchungen.")
else:
    print(f"Buchungen von {selected_guest.first_name} {selected_guest.last_name}:")
    selected_booking = choose_option(bookings, lambda b: f"ID: {b.booking_id}, Zimmer: {b.room_id}, {b.check_in_date} bis {b.check_out_date}, Storniert: {b.is_cancelled}", "Wähle eine Buchung aus")

    # 3. Buchung stornieren
    confirm = input(f"Möchtest du die Buchung ID {selected_booking.booking_id} wirklich stornieren? (j/n): ").lower()
    if confirm == 'j':
        success = booking_manager.update_booking(selected_booking.booking_id, is_cancelled=True)
        if success:
            print("Buchung erfolgreich storniert.")
        else:
            print("Fehler beim Stornieren der Buchung.")
    else:
        print("Stornierung abgebrochen.")


User-Story 7: Als Gast möchte ich eine dynamische Preisgestaltung auf der Grundlage der Nachfrage sehen, damit ich ein Zimmer zum besten Preis buchen kann.

In [ ]:
room_id = int(input("Insert room_id, from the room you want to book"))
check_in_date_str = input("Insert check in date in the following format YYYY-MM-DD")
check_out_date_str = input("Insert check out date in the following format YYYY-MM-DD")

total_price = room_manager.calculate_total_price_per_stay(room_id, check_in_date_str, check_out_date_str)

print(f"The total for your stay is going to be {total_price} CHF.")

User-Story 8: Als Admin des Buchungssystems möchte ich alle Buchungen aller Hotels sehen können, um eine Übersicht zu erhalten.

In [ ]:
username = input("Enter your username: ") # The username is "Admin"
password = input("Enter password: ") # The password is "FHNW"

if user_manager.is_admin(username, password):
    bookings = booking_manager.get_all_bookings()
    if bookings:
        print("All bookings in the system:")
        for booking in bookings:
            print(
                f"Booking_id: {booking['booking_id']}, "
                f"Guest: {booking['guest_id']}, "
                f"Room_id: {booking['room_id']}, "
                f"Check-in: {booking['check_in_date']}, "
                f"Check-out: {booking['check_out_date']}, "
                f"Is cancelled: {booking['is_cancelled']}, "
                f"Total amount: {booking['total_amount']} CHF"
            )
    else:
        print("No bookings found.")

else:
    print("Sie haben keine Zugriffsrechte.")

User-Story 9: Als Admin möchte ich eine Liste der Zimmer mit ihrer Ausstattung sehen, damit ich sie besser bewerben kann.

In [ ]:
username = input("Enter your username: ") # The username is "Admin"
password = input("Enter password: ") # The password is "FHNW"

if user_manager.is_admin(username, password):
    rooms_data = room_manager.get_all_rooms_details_with_facilities()

    if rooms_data:
        print("All rooms with facilities:")
        for room in rooms_data:
            print(
                f"Room_id: {room['room_id']}, "
                f"Room number: {room['room_number']}, "
                f"Room_type_id: {room['room_type']['type_id']}, "
                f"Facilities: {', '.join(room['facilities']) if 'facilities' in room and room['facilities'] else 'None'}"
            )
    else:
        print("No rooms found.")
        
else:
    print("Sie haben keine Zugriffsrechte.")

User-Story 10: Als Admin möchte ich in der Lage sein, Stammdaten zu verwalten, z.B. Zimmertypen, Einrichtungen, und Preise in Echtzeit zu aktualisieren, damit das Backend-System aktuelle Informationen hat.

In [ ]:
username = input("Enter your username: ") # The username is "Admin"
password = input("Enter password: ") # The password is "FHNW"

if user_manager.is_admin(username, password):
    
    print("Willkommen im Verwaltungsprogramm. Wo möchten Sie Änderungen vornehmen?")
    print("1: Hotel")
    print("2: Adresse")
    print("3: Room")
    print("4: Room Type")
    print("5: Facility")
    choose1 = input("Wählen Sie zwischen 1 und 5: ")

    if choose1 == "1":
        print("Was möchten Sie machen?:")
        print("1: Neues Hotel erstellen")
        print("2: Ein Hotel ändern")
        print("3: Ein Hotel löschen")
        choose_hotel = input("Wählen Sie Ihre gewünschte Aktion aus.")

        if choose_hotel == "1":
            print("Neue Hotels können nur in kombination mit einer neuen Addresse und Room erstellt werden.")

            street = input("Strasse und Hausnummer: ")
            city = input("Stadt: ")
            zip_code = input("PLZ: ")

            address = address_manager.add_address(street, city, zip_code)
            if address:
                print(f"Adresse erfolgreich angelegt: {street}, {zip_code} {city}")
                address_id = address[0]
            else:
                print("Fehler beim Anlegen der Adresse.")
                address_id = int(input("Bitte Address-ID manuell eingeben: "))

            hotel_name = input("Hotelname: ")
            stars = int(input("Sterne (1-5): "))

            hotel = hotel_manager.add_hotel(hotel_name, stars, address_id)
            if hotel:
                print(f"Hotel '{hotel_name}' erfolgreich angelegt!")
                hotel_id = hotel["hotel_id"]
            else:
                print("Fehler beim Anlegen des Hotels.")
                hotel_id = int(input("Bitte Hotel-ID manuell eingeben: "))
                
            room_number = input("Zimmernummer: ")
            room_type_id = int(input("Room-Type-ID: "))
            price_per_night = float(input("Preis pro Nacht: "))

            room = room_manager.add_room(room_number, hotel_id, room_type_id, price_per_night)
            if room:
                print(f"Zimmer {room_number} erfolgreich angelegt!")
            else:
                print("Fehler beim Anlegen des Zimmers.")

        if choose_hotel == "2":
            hotel_id_str = input("Welche hotel_id möchtest du bearbeiten: ")
            if not hotel_id_str:
                print("Keine Hotel ID eingegeben. Aktualisierung beendet.")
            else:
                hotel_id = int(hotel_id_str)
                name = input("Neuen Namen eingeben. Leerlassen falls keine Änderungen gewünscht sind: ")
                stars_str = input("Neue Anzahl Sterne (1-5). Leerlassen falls keine Änderungen gewünscht sind: ")
                address_id_str = input("Neue address_id. Leerlassen falls keine Änderungen gewünscht sind: ")
                stars = int(stars_str) if stars_str else None
                address_id = int(address_id_str) if address_id_str else None

                result = hotel_manager.update_hotel(
                    hotel_id,
                    name=name if name else None,
                    stars=stars,
                    address_id=address_id
                )
                if result:
                    print("Hotel erfolgreich aktualisiert!")
                else:
                    print("Hotel konnte nicht geändert werden.")
    
        if choose_hotel == "3":
            hotel_id_str = input("Welche hotel_id möchten Sie löschen: ")
            hotel_id = int(hotel_id_str) if hotel_id_str else None

            if hotel_id:
                result = hotel_manager.delete_hotel(hotel_id)
                if result:
                    print(f"Hotel mit hotel_id {hotel_id} gelöscht!")
                else:
                    print(f"Hotel mit hotel_id {hotel_id} konnte nicht gelöscht werden.")

            if not hotel_id:
                print("Keine ID eingegeben.")
    
    elif choose1 == "2":
        print("Was möchten Sie machen?:")
        print("1: Neue Addresse erstellen")
        print("2: Eine Addresse ändern")
        print("3: Eine Addresse löschen")
        choose_address = input("Wählen Sie Ihre gewünschte Aktion aus.")

        if choose_address == "1":
            street = input("Strasse und Hausnummer: ")
            city = input("Stadt: ")
            zip_code = input("PLZ: ")

            address = address_manager.add_address(street, city, zip_code)
            if address:
                print(f"Adresse erfolgreich angelegt: {street}, {zip_code} {city}")
                address_id = address[0]
            else:
                print("Fehler beim Anlegen der Adresse.")
                address_id = int(input("Bitte Address-ID manuell eingeben: "))        

        if choose_address == "2":        
            address_id_str = input("address_id: ")
            if not address_id_str:
                print("Keine Address ID eingegeben. Aktualisierung beendet.")
            else:
                address_id = int(address_id_str)
                street = input("new street name and number, otherwise leave empty: ")
                city = input("new city name, otherwise leave empty: ")
                zip_code_str = input("new zip code, otherwise leave empty: ")
                zip_code = int(zip_code_str) if zip_code_str else None

                result = address_manager.update_address(
                    address_id,
                    street=street if street else None,
                    city=city if city else None,
                    zip_code=zip_code
                )
                if result:
                    print("Addresse erfolgreich geändert!")
                else:
                    print("Addresse konnte nicht aktualisiert werden.")
        
        if choose_address == "3":
            address_id_str = input("Welche address_id möchten Sie löschen: ")
            if not address_id_str:
                print("Keine Address ID eingegeben. Aktualisierung beendet.")
            else:
                address_id = int(address_id_str)
                result = address_manager.delete_address(address_id)
                if result:
                    print(f"Addresss mit address_id {address_id} gelöscht!")
                else:
                    print(f"Addresss mit address_id {address_id} konnte nicht gelöscht werden.")

    elif choose1 == "3":
        print("Was möchten Sie machen?:")
        print("1: Neuen Room erstellen")
        print("2: Einen Room ändern")
        print("3: Einen Room löschen")
        choose_room = input("Wählen Sie Ihre gewünschte Aktion aus.")

        if choose_room == "1":
            room_number = input("Zimmernummer: ")
            room_type_id = int(input("Room-Type-ID: "))
            price_per_night = float(input("Preis pro Nacht: "))

            room = room_manager.add_room(room_number, hotel_id, room_type_id, price_per_night)
            if room:
                print(f"Zimmer {room_number} erfolgreich angelegt!")
        else:
            print("Fehler beim Anlegen des Zimmers.")


        if choose_room == "2":
            room_id_str = input("room_id: ")
            if not room_id_str:
                print("Keine Room ID eingegeben. Aktualisierung beendet.")
            else:
                room_id = int(room_id_str)
                room_number = input("Neue Room Nummer eingeben, sonst leer lassen: ")
                hotel_id_str = input("Neue Hotel ID eingeben, sonst leer lassen: ")
                room_type_id_str = input("Neue Room Type ID eingeben, sonst leer lassen: ")
                price_per_night_str = input("Neuer Preis pro Nacht eingeben, sonst leer lassen: ")

                hotel_id = int(hotel_id_str) if hotel_id_str else None
                room_type_id = int(room_type_id_str) if room_type_id_str else None
                price_per_night = float(price_per_night_str) if price_per_night_str else None

                result = room_manager.update_room(
                    room_id,
                    room_number if room_number else None,
                    hotel_id,
                    room_type_id,
                    price_per_night
                )
                if result:
                    print("Room erfolgreich geändert!")
                else:
                    print("Fehler beim ändern des Room.")
        
        if choose_room == "3":
            room_id_str = input("Welche room_id möchten Sie löschen: ")
            if not room_id_str:
                print("Keine Room ID eingegeben. Aktualisierung beendet.")
            else:
                room_id = int(room_id_str)
                result = room_manager.delete_room(room_id)
                if result: 
                    print(f"Zimmer mit room_id {room_id} gelöscht!")
                else:
                    print(f"Zimmer mit room_id {room_id} konnte nicht gelöscht werden.")

    elif choose1 == "4":
        print("Was möchten Sie machen?:")
        print("1: Neuen Room Type erstellen")
        print("2: Einen Room Type ändern")
        print("3: Einen Room Type löschen")
        choose_room_type = input("Wählen Sie Ihre gewünschte Aktion aus.")
            
        if choose_room_type == "1":    
            description = input("Beschreibung des Room Types: ")
            max_guests_str = input("Maximale Anzahl Gäste: ")
            max_guests = int(max_guests_str) if max_guests_str else None

            room_type = room_type_manager.add_room_type(description, max_guests)
            if room_type:
                print(f"Room Type '{description}' erfolgreich erstellt!")
            else:
                print("Fehler beim erstellen des Room Types.")
        
        if choose_room_type == "2":
            room_type_id_str = input("room_type_id: ")
            if not room_type_id_str:
                print("Keine Room Type ID eingegeben. Aktualisierung beendet.")
            else:
                room_type_id = int(room_type_id_str)
                description = input("Neue Beschreibung eingeben, sonst leer lassen: ")
                max_guests_str = input("Neue maximale Anzahl Gäste eingeben, sonst leer lassen: ")
                max_guests = int(max_guests_str) if max_guests_str else None

                result = room_type_manager.update_room_type(
                    room_type_id,
                    description=description if description else None,
                    max_guests=max_guests
                )
                if result:
                    print("Room Type erfolgreich geändert!")
                else:
                    print("Fehler beim ändern des Room Type.")
        
        if choose_room_type == "3":
            room_type_id_str = input("Welche room_type_id möchten Sie löschen: ")
            if not room_type_id_str:
                print("Keine Room Type ID eingegeben. Aktualisierung beendet.")
            else:
                room_type_id = int(room_type_id_str)
                result = room_type_manager.delete_room_type(room_type_id)
                if result:
                    print(f"Room Type mit room_type_id {room_type_id} gelöscht!")
                else:
                    print(f"Room Type mit room_type_id {room_type_id} konnte nicht gelöscht werden.")
    
    elif choose1 == "5":
        print("Was möchten Sie machen?:")
        print("1: Neue Facility erstellen")
        print("2: Eine Facility ändern")
        print("3: Eine Facility löschen")
        choose_facility = input("Wählen Sie Ihre gewünschte Aktion aus.")

        if choose_facility == "1":
            description = input("Facility beschreiben: ")
            facility = facility_manager.add_facility(description)
            if facility:
                print(f"Facility '{description}' erfolgreich erstellt!")
            else:
                print("Fehler beim erstellen der Facility.")
        
        if choose_facility == "2":
            facility_id_str = input("facility_id: ")
            if not facility_id_str:
                print("Keine Facility ID eingegeben. Aktualisierung beendet.")
            else:
                facility_id = int(facility_id_str)
                description = input("Neue Beschreibung eingeben, sonst leer lassen: ")

                result = facility_manager.update_facility(facility_id,description=description if description else None)
                if result:
                    print("Facility erfolgreich geändert!")
                else:
                    print("Fehler beim ändern der Facility.")
            
        if choose_facility == "3":
            facility_id_str = input("Welche facility_id möchten Sie löschen: ")
            
            if not facility_id_str:
                print("Keine Facility ID eingegeben. Aktualisierung beendet.")
            
            else:
                facility_id = int(facility_id_str)
                result = facility_manager.delete_facility(facility_id)
                if result:
                    print(f"Facility mit facility_id {facility_id} gelöscht!")
                else:
                    print(f"Facility mit facility_id {facility_id} konnte nicht gelöscht werden.")

else:
    print("Sie haben keine Zugriffsrechte.")
    